### 모델 라이브러리

In [1]:
# 모델 라이브러리
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

# 스테킹
from sklearn.ensemble import StackingClassifier

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

def objectiveRF(trial, x_tr, y_tr, x_val, y_val):
    param = {
    'n_estimators' : trial.suggest_int('n_estimators', 1000, 2000),
    'max_depth' : trial.suggest_int('max_depth', 30, 50),
    'min_samples_split' : trial.suggest_int('min_samples_split', 2, 10),
    'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 10),
    'criterion' : trial.suggest_categorical("criterion", ["gini", "entropy",]),
    'class_weight' : trial.suggest_categorical("class_weight", ["balanced"]),
    }
    
    model = RandomForestClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    df_train_encoded.drop("is_converted", axis=1),
    df_train_encoded["is_converted"].astype(int),
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveRF(trial, x_train, y_train, x_val, y_val), n_trials=100)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


[I 2024-02-25 02:50:36,719] A new study created in memory with name: no-name-d0ee3272-0993-4a0c-8ff7-d3dc6d22f1c0
[I 2024-02-25 02:52:12,781] Trial 0 finished with value: 0.8134642356241234 and parameters: {'n_estimators': 1549, 'max_depth': 45, 'min_samples_split': 7, 'min_samples_leaf': 6, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.8134642356241234.
[I 2024-02-25 02:53:39,533] Trial 1 finished with value: 0.8186968838526912 and parameters: {'n_estimators': 1438, 'max_depth': 48, 'min_samples_split': 10, 'min_samples_leaf': 4, 'criterion': 'gini', 'class_weight': 'balanced'}. Best is trial 1 with value: 0.8186968838526912.
[I 2024-02-25 02:55:25,185] Trial 2 finished with value: 0.8376753507014028 and parameters: {'n_estimators': 1568, 'max_depth': 49, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}. Best is trial 2 with value: 0.8376753507014028.
[I 2024-02-25 02:57:14,792] Trial 3 finished wit

Best trial: score 0.8402603905858789, 
params {'n_estimators': 1935, 'max_depth': 32, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy', 'class_weight': 'balanced'}


### 모델 성능확인

In [3]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)

### 모델 정의

Optuna를 이용하여 파라미터를 구함

In [2]:
# CatBoost
model = RandomForestClassifier(
    n_estimators=1935
    , max_depth=32
    , min_samples_split=2
    , min_samples_leaf=1   
    , bootstrap=True
    , criterion='entropy'
    , class_weight='balanced'
    , random_state=0
)

### 모델 학습

In [72]:
model.fit(x_train, y_train)

In [73]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)


오차행렬:


,True,False
True,747,200
False,78,10835


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.97656,0.905455,0.788807,0.843115,0.975818


In [74]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

598

### 제출파일 생성

In [75]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_rf_optuna.csv", index=False)

.